In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
trial_folder = "C:/Users/nicho/Desktop/Thesis Work/11-11-25/"  # repalce with folder path of where data should go
sensors_used = ["A5F2", "A19E"]       # replace with name of raw data file

In [ ]:
# dictionary to hold sensor name and dataframe associated
dfs = {}
latest_starting_time = None
earliest_ending_time = None

for sensor in sensors_used:
    df_raw = pd.read_csv(trial_folder + sensor + ".csv", sep='\t', skiprows=1)
    df_raw = df_raw.drop(columns=[col for col in df_raw.columns if "Unnamed" in col], errors='ignore')

    # first row contains units — remove it and reset index
    df = df_raw.iloc[1:].reset_index(drop=True)


    # convert all columns to numeric
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    dfs[sensor] = df

    starting_time = df.iloc[0][f'Shimmer_{sensor}_TimestampSync_Unix_CAL']
    ending_time = df.iloc[-1][f'Shimmer_{sensor}_TimestampSync_Unix_CAL']
    
    if latest_starting_time == None or starting_time > latest_starting_time:
        latest_starting_time = starting_time
        latest_start_sensor = sensor
    
    if earliest_ending_time == None or ending_time < earliest_ending_time:
        earliest_ending_time = ending_time
        earliest_end_sensor = sensor
print(latest_start_sensor)
print(earliest_end_sensor)

In [ ]:
master_sensor = None
if earliest_end_sensor == latest_start_sensor:
    master_sensor = earliest_end_sensor
else:
    latest_start_df = dfs[latest_start_sensor]
    dfs[latest_start_sensor] = latest_start_df[latest_start_df[f'Shimmer_{latest_start_sensor}_TimestampSync_Unix_CAL'] <= earliest_ending_time]
    master_sensor = latest_start_sensor

nonmaster_sensor = None
for sensor in sensors_used:
    if sensor != master_sensor:
        nonmaster_sensor = sensor
print(master_sensor)
print(nonmaster_sensor)

In [ ]:
print(master_sensor)
print(nonmaster_sensor)
for sensor in sensors_used:
    print(sensor)
    print(dfs[sensor].iloc[0][f'Shimmer_{sensor}_TimestampSync_Unix_CAL'])
    print(dfs[sensor].iloc[-1][f'Shimmer_{sensor}_TimestampSync_Unix_CAL'])

In [ ]:
import pandas as pd

def merge_shimmers(
    df_master: pd.DataFrame,
    t_master: str,
    master_name: str,
    df_other: pd.DataFrame,
    t_other: str,
    other_name: str,
    tolerance: float = 10,   # milliseconds
    keep_other_time: bool = False
) -> pd.DataFrame:
    """
    Align (snap) df_other to df_master's timeline using nearest-neighbor on timestamps,
    then return a single DataFrame on the master time grid.

    Parameters
    ----------
    df_master : DataFrame
        The 'master' sensor data. Its time column defines the output timeline.
    t_master : str
        Name of the master time column.
    df_other : DataFrame
        The other sensor's data to be aligned to the master timeline.
    t_other : str
        Name of the other sensor's time column
    tolerance : float, default 10ms
        Max allowed absolute time difference  for a match. Rows without a
        nearby sample within this window remain NaN for the other sensor's columns.
        For 50 Hz (20 ms period), ±10 ms is a good default.
    keep_other_time : bool, default False
        If True, keep df_other's time column (it will be renamed with the prefix).
        If False, drop it (you'll have a single, master time column).

    Returns
    -------
    DataFrame
        df_master columns (unchanged) + df_other columns (prefixed), aligned to the
        master timeline, with a single master time column unless keep_other_time=True.
    """
    # Safety: merge_asof requires sorted keys
    left = df_master.sort_values(t_master).reset_index(drop=True)
    left = left.rename(columns={f'Shimmer_{master_name}_Event_Marker_CAL': 'Event_Marker'})
    right = df_other.sort_values(t_other).reset_index(drop=True)
    right = right.drop(f'Shimmer_{other_name}_Event_Marker_CAL', axis=1)
    
    merged = pd.merge_asof(
        left,
        right,
        left_on=t_master,
        right_on=t_other,
        direction='nearest',
        tolerance=tolerance,
    )

    if not keep_other_time and t_other in merged.columns:
        merged = merged.drop(columns=[t_other])


    return merged


In [ ]:
merged = merge_shimmers(df_master=dfs[master_sensor], 
                        t_master=f'Shimmer_{master_sensor}_TimestampSync_Unix_CAL',
                        master_name=master_sensor, 
                        df_other=dfs[nonmaster_sensor],
                        t_other=f'Shimmer_{nonmaster_sensor}_TimestampSync_Unix_CAL',
                        other_name=nonmaster_sensor,
                        tolerance=10,
                        keep_other_time=False
                        )



# convert timestamp to relative time (seconds)
if f'Shimmer_{master_sensor}_TimestampSync_Unix_CAL' in merged.columns:
    merged['Time (s)'] = (merged[f'Shimmer_{master_sensor}_TimestampSync_Unix_CAL'] - merged[f'Shimmer_{master_sensor}_TimestampSync_Unix_CAL'].iloc[0]) / 1000

# drop unwanted columns
drop_cols = []
for sensor_id in sensors_used:
    drop_cols.append(f"Shimmer_{sensor_id}_Battery_CAL")
    drop_cols.append(f"Shimmer_{sensor_id}_ECG_EMG_Status1_CAL")
    drop_cols.append(f'Shimmer_{sensor_id}_TimestampSync_Unix_CAL')
merged = merged.drop(columns=[c for c in drop_cols if c in merged.columns], errors='ignore')

rename_map = {}
for sensor_id in sensors_used:
    rename_map.update({
        f'Shimmer_{sensor_id}_Accel_LN_X_CAL': f'{sensor_id}_AccelX',
        f'Shimmer_{sensor_id}_Accel_LN_Y_CAL': f'{sensor_id}_AccelY',
        f'Shimmer_{sensor_id}_Accel_LN_Z_CAL': f'{sensor_id}_AccelZ',
        f'Shimmer_{sensor_id}_EMG_CH1_24BIT_CAL': f'{sensor_id}_EMG1',
        f'Shimmer_{sensor_id}_EMG_CH2_24BIT_CAL': f'{sensor_id}_EMG2',
        f'Shimmer_{sensor_id}_Gyro_X_CAL': f'{sensor_id}_GyroX',
        f'Shimmer_{sensor_id}_Gyro_Y_CAL': f'{sensor_id}_GyroY',
        f'Shimmer_{sensor_id}_Gyro_Z_CAL': f'{sensor_id}_GyroZ',
        f'Shimmer_{sensor_id}_Mag_X_CAL': f'{sensor_id}_MagX',
        f'Shimmer_{sensor_id}_Mag_Y_CAL': f'{sensor_id}_MagY',
        f'Shimmer_{sensor_id}_Mag_Z_CAL': f'{sensor_id}_MagZ',
    })
merged = merged.rename(columns=rename_map)

merged.to_csv(trial_folder + "merged.csv", index=False)

In [ ]:
# %%
# helper function to create interactive line plots
def plot_group(df, group_cols, title):
    fig = go.Figure()
    for col in group_cols:
        fig.add_trace(go.Scatter(x=df['Time (s)'], y=df[col], mode='lines', name=col))
    fig.update_layout(
        title=title,
        xaxis_title="Time (s)",
        yaxis_title="Sensor Value",
        hovermode="x unified",
        template="plotly_white"
    )
    fig.show()

# %%
# identify column groups
emg_cols = [c for c in merged.columns if "EMG" in c]
accel_cols = [c for c in merged.columns if "Accel" in c]
gyro_cols = [c for c in merged.columns if "Gyro" in c]
mag_cols = [c for c in merged.columns if "Mag" in c]

# %%
# --- plot each sensor group ---
if emg_cols:
    plot_group(merged, emg_cols, "EMG Channels")

if accel_cols:
    plot_group(merged, accel_cols, "Accelerometer (X, Y, Z)")

if gyro_cols:
    plot_group(merged, gyro_cols, "Gyroscope (X, Y, Z)")

if mag_cols:
    plot_group(merged, mag_cols, "Magnetometer (X, Y, Z)")

In [ ]:
df = pd.read_csv(trial_folder + "merged.csv")
print(df.columns)
print(df.iloc[-1]['Time (s)'])

In [ ]:
start_time = df.loc[df['Event_Marker'] > 0, 'Time (s)'].iloc[0]
end_time = df.loc[df['Event_Marker'] > 0, 'Time (s)'].iloc[-1]
print(start_time)
print(end_time)

In [ ]:
#start_time = 35.7979
trimmed_video_len = 492.71

In [ ]:
def trim_df(df, start_time, total_time):
    """
    Trim the dataframe to only include rows where 'Time (s)' 
    is between start_time and total_time + start_time.
    Then reset time so it starts at 0.
    """
    trimmed_df = df[(df["Time (s)"] >= start_time) & (df["Time (s)"] <= total_time + start_time)].copy()
    trimmed_df["Time (s)"] = trimmed_df["Time (s)"] - start_time
    trimmed_df.reset_index(drop=True, inplace=True)
    return trimmed_df

In [ ]:
# trim dataframe to sync with video
df = trim_df(df, start_time, trimmed_video_len) 
print(df['Time (s)'])

In [ ]:
import cv2

video_path = trial_folder + "trimmed_finalized.mp4"

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
cap.release()

print(f"FPS: {fps}")

In [ ]:
total_frames = 54149
json_path = trial_folder + "labels" + ".json"

In [ ]:
df = pd.read_csv(trial_folder + "merged.csv")

In [ ]:
def get_fps(df, frames):
    start = df.iloc[0]['Time (s)']
    end = df.iloc[-1]['Time (s)']
    dt = end - start
    return frames / dt

In [ ]:
import json

# load JSON labels
with open(json_path, "r") as f:
    data = json.load(f)

# collect start/end frames per annotation
annotations_list = []
for task in data:
    for annotation in task.get("annotations", []):
        for result in annotation.get("result", []):
            label = result["value"]["timelinelabels"][0]
            start = result["value"]["ranges"][0]["start"]
            end = result["value"]["ranges"][0]["end"]

            annotations_list.append({
                "label": label,
                "start_frame": start,
                "end_frame": end
            })

# get fps of video
fps = get_fps(df, total_frames)

for item in annotations_list:
    label = item['label']
    start_time = (item['start_frame'] - 1) / fps
    end_time = (item['end_frame'] - 1) / fps
    df.loc[(df["Time (s)"] >= start_time) & (df["Time (s)"] <= end_time), "Primitive"] = label

    mask = (df["Time (s)"] >= start_time) & (df["Time (s)"] <= end_time)
    indices = df.index[mask]

    if not indices.empty:
        df.loc[indices[0], "Primitive"] = "Start"
        df.loc[indices[-1], "Primitive"] = "End"


In [ ]:
import numpy as np
from scipy.signal import butter, sosfiltfilt, iirnotch, filtfilt


# gather list of emg columns
emg_columns = []
for sensor_id in sensors_used:
    emg_columns.append(f'{sensor_id}_EMG1')
    emg_columns.append(f'{sensor_id}_EMG2')


# get sampling rate from time differences
dt = np.diff(df["Time (s)"].to_numpy())
fs = float(np.round(1.0/np.median(dt[dt>0])))
nyq = 0.5*fs

# bandpass filter: keeps the core EMG frequency band where motor-unit signals live and rejects everything else
def bp_sos(low, high, order=4):
    return butter(order, [low/nyq, high/nyq], btype='bandpass', output='sos')

# high pass filter: removes slow drift and movement artifacts so only muscle activity remains
def hp_sos(cut, order=2):
    return butter(order, cut/nyq, btype='highpass', output='sos')

# narrow notch filter to remove power line interference
def notch_once(x, f0, Q=35):
    if f0 >= nyq: return x
    w0 = f0/(fs/2); b, a = iirnotch(w0, Q)
    return filtfilt(b, a, x)

# linear envelope: full-wave rectification followed by low-pass filter to smooth
def linear_envelope(x, lp=5.0):
    rect = np.abs(x)
    sos_lp = butter(4, lp/nyq, btype='lowpass', output='sos')
    return sosfiltfilt(sos_lp, rect)

# filters
sos_hp = hp_sos(20.0, order=2)
sos_bp = bp_sos(20.0, min(240.0, 0.45*fs), order=4)

for col in emg_columns:
    x = df[col].astype(float).interpolate('linear').to_numpy()

    # high pass
    x = sosfiltfilt(sos_hp, x)

    # notch 60 & 120
    x = notch_once(x, 60); x = notch_once(x, 120)

    # bandpass
    x_bp = sosfiltfilt(sos_bp, x)

    # envelope
    env = linear_envelope(x_bp, lp=5.0)

    # save both normally filtered and enveloped signal
    df[col] = x_bp
    df[col + "_ENV"] = env

In [ ]:
import numpy as np

def estimate_fs_modality(df, axes, time_col="Time (s)", eps=1e-6):
    """
    Estimate IMU sampling rate for a modality (e.g., ['AccelX','AccelY','AccelZ'])
    by detecting rows where ANY axis changes value. Returns a float fs_est (Hz).
    """
    t = df[time_col].to_numpy()
    X = df[axes].to_numpy(dtype=float)
    # mark where any axis changes between rows
    change = (np.abs(np.diff(X, axis=0)) > eps).any(axis=1)
    change_times = t[1:][change]
    if change_times.size < 2:
        return None
    dt = np.diff(change_times)
    dt = dt[dt > 0]
    if dt.size == 0:
        return None
    fs_est = 1.0 / np.median(dt)  # robust estimate
    return float(fs_est)

fs_accel = estimate_fs_modality(df, [f'{sensor_id}_AccelX',f'{sensor_id}_AccelX',f'{sensor_id}_AccelZ'])
fs_gyro  = estimate_fs_modality(df, [f'{sensor_id}_GyroX',f'{sensor_id}_GyroY',f'{sensor_id}_GyroZ'])
fs_mag   = estimate_fs_modality(df, [f'{sensor_id}_MagX',f'{sensor_id}_MagY',f'{sensor_id}_MagZ'])
print(fs_accel)
print(fs_gyro)
print(fs_mag)


In [ ]:
from scipy.signal import butter, sosfiltfilt

# -- IMU FILTER HELPERS --

# lowpass filter
def lowpass_sos(data, cutoff, fs, order=4):
    nyq = 0.5 * fs
    sos = butter(order, cutoff/nyq, btype="low", output="sos")
    return sosfiltfilt(sos, data)

def highpass_sos(data, cutoff, fs, order=2):
    nyq = 0.5 * fs
    sos = butter(order, cutoff/nyq, btype="highpass", output="sos")
    return sosfiltfilt(sos, data)

for col in accel_cols:
    df[col + "_DYN"] = highpass_sos(df[col], cutoff=0.3, fs=50)


In [ ]:
# gather list of accelerometer columns
accel_cols = []
for sensor_id in sensors_used:
    accel_cols.append(f'{sensor_id}_AccelX')
    accel_cols.append(f'{sensor_id}_AccelY')
    accel_cols.append(f'{sensor_id}_AccelZ')

for col in accel_cols:
    df[col] = lowpass_sos(df[col], cutoff=20, fs=fs_accel)
    df[col + "_DYN"] = highpass_sos(df[col], cutoff=0.3, fs=50) # cut out gravity with high pass filter

In [ ]:
# gather list of accelerometer columns
gyro_cols = []
for sensor_id in sensors_used:
    gyro_cols.append(f'{sensor_id}_GyroX')
    gyro_cols.append(f'{sensor_id}_GyroY')
    gyro_cols.append(f'{sensor_id}_GyroZ')

for col in gyro_cols:
    df[col] = lowpass_sos(df[col], cutoff=20, fs=fs_gyro)

In [ ]:
# gather list of accelerometer columns
mag_cols = []
for sensor_id in sensors_used:
    mag_cols.append(f'{sensor_id}_MagX')
    mag_cols.append(f'{sensor_id}_MagY')
    mag_cols.append(f'{sensor_id}_MagZ')

for col in mag_cols:
    df[col] = lowpass_sos(df[col], cutoff=10, fs=fs_mag)

In [ ]:
import numpy as np

for sensor_id in sensors_used:
    df[f"{sensor_id}_AccelMag"] = np.sqrt(df[f'{sensor_id}_AccelX']**2 + df[f'{sensor_id}_AccelY']**2 + df[f'{sensor_id}_AccelZ']**2)
    df[f"{sensor_id}_GyroMag"]  = np.sqrt(df[f'{sensor_id}_GyroX']**2  + df[f'{sensor_id}_GyroY']**2  + df[f'{sensor_id}_GyroZ']**2)
    df[f"{sensor_id}_MagnetMag"]   = np.sqrt(df[f'{sensor_id}_MagX']**2   + df[f'{sensor_id}_MagY']**2   + df[f'{sensor_id}_MagZ']**2)


In [ ]:
for col in accel_cols + gyro_cols + mag_cols:
    mu, sigma = df[col].mean(), df[col].std()
    df[col + "_Z"] = (df[col] - mu) / (sigma + 1e-8)

for sensor_id in sensors_used:
    for col in [f"{sensor_id}_AccelMag"] + [f"{sensor_id}_GyroMag"] + [f"{sensor_id}_MagnetMag"]:
        mu, sigma = df[col].mean(), df[col].std()
        df[col + "_Z"] = (df[col] - mu) / (sigma + 1e-8)

In [ ]:
df.to_csv(trial_folder + "data_labeled" + ".csv", index=False)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# --- Define channels dynamically based on sensor_ids ---
channels = {}
for sensor_id in sensors_used:
    channels.update({
        f"{sensor_id} EMG1": f"{sensor_id}_EMG1_ENV",
        f"{sensor_id} EMG2": f"{sensor_id}_EMG2_ENV",
        f"{sensor_id} Accel X": f"{sensor_id}_AccelX",
        f"{sensor_id} Accel Y": f"{sensor_id}_AccelY",
        f"{sensor_id} Accel Z": f"{sensor_id}_AccelZ",
        f"{sensor_id} Gyro X": f"{sensor_id}_GyroX",
        f"{sensor_id} Gyro Y": f"{sensor_id}_GyroY",
        f"{sensor_id} Gyro Z": f"{sensor_id}_GyroZ",
        f"{sensor_id} Mag X": f"{sensor_id}_MagX",
        f"{sensor_id} Mag Y": f"{sensor_id}_MagY",
        f"{sensor_id} Mag Z": f"{sensor_id}_MagZ",
    })

# --- Build contiguous primitive segments ---
df["Primitive_Change"] = (df["Primitive"] != df["Primitive"].shift()).cumsum()

# --- Create a large distinct color palette ---
distinct_colors = (
    px.colors.qualitative.Bold +
    px.colors.qualitative.Safe +
    px.colors.qualitative.Dark24 +
    px.colors.qualitative.Light24
)

# --- Assign colors to primitives ---
unique_primitives = df["Primitive"].unique()
primitive_color_map = {p: distinct_colors[i % len(distinct_colors)] for i, p in enumerate(unique_primitives)}

# --- Create subplots ---
fig = make_subplots(
    rows=len(channels),
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.02,
    subplot_titles=list(channels.keys())
)

# --- Plot each channel ---
for i, (name, col) in enumerate(channels.items(), start=1):
    if col not in df.columns:
        continue

    added_labels = set()  # track which primitives have been added to legend

    for _, seg in df.groupby("Primitive_Change"):
        primitive = seg["Primitive"].iloc[0]
        color = primitive_color_map.get(primitive, "blue")

        # Show legend only for first subplot
        show_legend = (i == 1) and (primitive not in added_labels)

        # --- Plot main signal segment ---
        fig.add_trace(
            go.Scatter(
                x=seg["Time (s)"] if "Time (s)" in df.columns else seg.index,
                y=seg[col],
                mode="lines",
                line=dict(color=color, width=1.5),
                name=primitive,
                showlegend=show_legend,
                hovertemplate=f"<b>{name}</b><br>Primitive: {primitive}<br>Time: %{{x}}<br>Value: %{{y}}<extra></extra>"
            ),
            row=i,
            col=1
        )
        added_labels.add(primitive)

# --- Layout ---
fig.update_layout(
    height=300*len(channels),  # dynamic height
    showlegend=True,
    legend_title="Primitive / Marker",
    title="Shimmer Sensor Channel Dashboard",
    hovermode="x unified",
    margin=dict(t=50, b=30, l=30, r=30),
    template="plotly_white"
)

fig.show()


